# Part III (Static Replication)

## 1. PRICE

> payoff function

$$
payoff = S_T^{1/3} + 1.5 \times \log(S_T) + 10.0
$$

>the price

$$
V_0 = e^{-rT}E[V_T]
$$

In [90]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import datetime as dt
import array
from scipy.stats import norm
from scipy.optimize import brentq,fsolve
from scipy import interpolate
from scipy.optimize import least_squares

### calculate sigma

In [92]:
spx_df = pd.read_csv('SPX_options.csv')
rate_df = pd.read_csv('zero_rates_20201201.csv')
spx_df['strike_price'] = spx_df['strike_price']/1000
spx_df['mid_price'] = (spx_df['best_bid'] + spx_df['best_offer'])/2

In [94]:
# Black-Scholes Model
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

# Bachelier Model
def BachelierCall(S, K, r, sigma, T):
    d = (S-K) / (S*sigma*np.sqrt(T))
    disc = np.exp(-r*T)
    return disc*((S-K)*norm.cdf(d)+S*sigma*np.sqrt(T)*norm.pdf(d))

def BachelierPut(S, K, r, sigma, T):
    d = (S-K) / (S*sigma*np.sqrt(T))
    disc = np.exp(-r*T)
    return disc*((K-S)*norm.cdf(-d)+S*sigma*np.sqrt(T)*norm.pdf(-d))

In [132]:
# Implied European Options Volatility Model

# SPX Maturity Data
spx1 = spx_df[(spx_df.exdate == 20201218)]
spx2 = spx_df[(spx_df.exdate == 20210115)]
spx3 = spx_df[(spx_df.exdate == 20210219)]

# Time To Maturity
today = dt.date(2020, 12, 1)
exdate1 = dt.date(2020, 12, 18)
exdate2 = dt.date(2021, 1, 15)
exdate3 = dt.date(2021, 2, 19)
T1 = (exdate1-today).days/365.0
T2 = (exdate2-today).days/365.0
T3 = (exdate3-today).days/365.0

# Discount Rate Interpolation
x = rate_df['days']
y = rate_df['rate']
f = interpolate.interp1d(x,y)
r1 = f(T1*365)/100
r2 = f(T2*365)/100
r3 = f(T3*365)/100

# Underlying Value & ATM Strike Price
S = 3662.45
K = 3660

def impliedCallVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesCall(S, K, r, x, T),
                        1e-6, 10)
    except Exception:
        impliedVol = np.nan
    return impliedVol

def impliedPutVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 10)
    except Exception:
        impliedVol = np.nan
    return impliedVol

def impliedVolatility(S, K, r, price, T, option_type):
    try:
        if option_type == 'C':
            return brentq(lambda x: price - BlackScholesCall(S, K, r, x, T), 1e-12, 10)
        else:
            return brentq(lambda x: price - BlackScholesPut(S, K, r, x, T), 1e-12, 10)
    except Exception:
        return np.nan

def calculate_atm_volatility(spx_df, S, K, r, T):
    atm_call = spx_df[(spx_df['strike_price'] == K) & (spx_df['cp_flag'] == 'C')]
    atm_put = spx_df[(spx_df['strike_price'] == K) & (spx_df['cp_flag'] == 'P')]

    if not atm_call.empty and not atm_put.empty:
        sigma_call = impliedVolatility(S, K, r, atm_call.iloc[0]['mid_price'], T, 'C')
        sigma_put = impliedVolatility(S, K, r, atm_put.iloc[0]['mid_price'], T, 'P')
        return (sigma_call + sigma_put) / 2
    else:
        return np.nan

# calculate_atm_volatility
sigma1x = calculate_atm_volatility(spx1, S, K, r1, T1)
sigma2x = calculate_atm_volatility(spx2, S, K, r2, T2)
sigma3x = calculate_atm_volatility(spx3, S, K, r3, T3)

sigmax=[sigma1x ,sigma2x ,sigma3x ]
sigmax

[0.1750985704585497, 0.18537188428716733, 0.1909732726819407]

In [186]:
# Implied American Options Volatility Model
spy_df = pd.read_csv('SPY_options.csv')
spy_df['strike_price'] = spy_df['strike_price']/1000
spy_df['mid_price'] = (spy_df['best_bid'] + spy_df['best_offer'])/2

# SPX Maturity Data
spy1 = spy_df[(spy_df.exdate == 20201218)]
spy2 = spy_df[(spy_df.exdate == 20210115)]
spy3 = spy_df[(spy_df.exdate == 20210219)]

# Underlying Value & ATM Strike Price
S = 366.02
K = 366

# calculate_atm_volatility
sigma1y = calculate_atm_volatility(spy1, S, K, r1, T1)
sigma2y = calculate_atm_volatility(spy2, S, K, r2, T2)
sigma3y = calculate_atm_volatility(spy3, S, K, r3, T3)

sigmay=[sigma1y ,sigma2y ,sigma3y ]
sigmay

[0.186690316986162, 0.18481154419544626, 0.19108134783087416]

### Black-Scholes

$$
S_T = S_0 \exp \left( \left( r - \frac{\sigma^2}{2} \right)T + \sigma Z \right), \quad Z \sim N(0,1)
$$

In [301]:
def exotic_payoff_black_scholes(S0x, K, r, sigma, T):
    """
    Pricing exotic payoff under Black-Scholes model.

    Parameters:
        S0: float, initial stock price
        r: float, risk-free rate
        sigma: float, volatility
        T: float, time to maturity in years
    Returns:
        price: float, discounted payoff value
    """
    # Analytical expectations under Black-Scholes assumptions
    # Expectation of S_T^(1/3)
    term1 = S0x * np.exp((r - 0.5 * (sigma ** 2)) * T )

    # Expectation of log(S_T)
    term2 = np.log(S0x) + (r - 0.5 * sigma ** 2) * T

    # Payoff value
    payoff = term1 ** (1/3) + 1.5 * term2 + 10.0

    # Discount payoff to present value
    price = np.exp(-r * T) * payoff
    return price

# inputs
S0x = 3662.45  # Current stock price
r = r2  # Risk-free rate
sigma = sigma2x # Volatility
T = 45 / 365  # Time to maturity in years

# Compute price
price = exotic_payoff_black_scholes(S0x, None, r, sigma, T)
print(f"Under BlackScholes, the price of exotic European derivative (Payoff function): {price}")

Under BlackScholes, the price of exotic European derivative (Payoff function): 37.701220356138386


In [299]:
import numpy as np
from scipy.stats import norm

def exotic_payoff_black_scholes(S0y, K, r, sigma, T):
    """
    Pricing exotic payoff under Black-Scholes model.

    Parameters:
        S0: float, initial stock price
        r: float, risk-free rate
        sigma: float, volatility
        T: float, time to maturity in years
    Returns:
        price: float, discounted payoff value
    """
    # Analytical expectations under Black-Scholes assumptions
    # Expectation of S_T^(1/3)
    term1 = S0y * np.exp((r - 0.5 * (sigma ** 2)) * T )

    # Expectation of log(S_T)
    term2 = np.log(S0y) + (r - 0.5 * sigma ** 2) * T

    # Payoff value
    payoff = term1 ** (1/3) + 1.5 * term2 + 10.0

    # Discount payoff to present value
    price = np.exp(-r * T) * payoff
    return price

# inputs
S0y = 366.02  # Current stock price
r = r2  # Risk-free rate
sigma = sigma2y  # Volatility
T = 45 / 365  # Time to maturity in years

# Compute price
price = exotic_payoff_black_scholes(S0y, None, r, sigma, T)
print(f"Under BlackScholes, the price of exotic American derivative (Payoff function): {price}")


Under BlackScholes, the price of exotic American derivative (Payoff function): 25.993483630433552


### Bachelier

$$S_T = S_0 + \sigma W_T$$

In [112]:
from scipy.integrate import quad

In [295]:
# spx
def func(x):
    return (((S0x + sigma2x * np.sqrt(T) * x))**(1/3) + 1.5*(np.log(S0x + sigma2x * np.sqrt(T) * x)) + 10)\
    * (1 / np.sqrt(2 * np.pi))* np.exp(-0.5 * np.power(x, 2)) 
# 计算积分
a, b = -np.inf, np.inf # 积分上下限
result, error = quad(func, a, b)  # 结果和误差

print(f"Under Bachelier model, the price of exotic European derivative (Payoff function): {result}")

Under Bachelier model, the price of exotic European derivative (Payoff function): 37.723134748099234


In [297]:
# spy
def func(x):
    return (((S0y + sigma2y * np.sqrt(T) * x))**(1/3) + 1.5*(np.log(S0y + sigma2y * np.sqrt(T) * x)) + 10)\
    * (1 / np.sqrt(2 * np.pi))* np.exp(-0.5 * np.power(x, 2)) 
# 计算积分
a, b = -np.inf, np.inf # 积分上下限
result, error = quad(func, a, b)  # 结果和误差

print(f"Under Bachelier model, the price of exotic American derivative (Payoff function): {result}")

Under Bachelier model, the price of exotic American derivative (Payoff function): 26.007252302267414


### SABR

>The static replication price of the exotic payoff is given by:

$$
\text{Price} = e^{-rT} \cdot h(F) + \int_0^F \phi(K) P(F, K) \, dK + \int_F^\infty \phi(K) C(F, K) \, dK
$$

>Forward Price :

$$
   F = S \cdot e^{rT}
$$$$
   F = S \cdot e^{rT}
$$
   where \( S \) is the current spot price, \( r \) is the risk-free rate, and \( T \) is the time to expiry.

>Exotic Payoff at Forward Price :

$$
h(F) = F^{1/3} + 1.5 \cdot \log(F) + 10
$$

>Weight Function :

$$
   \phi(K) = -\frac{2}{9} K^{-5/3} - \frac{1.5}{K^2}
$$

> ATM comdition

$$
\sigma_{\text{imp}} = \frac{\alpha}{F^{1-\beta}} \left[1 + \left(\frac{(1-\beta)^2}{24} \cdot \frac{\alpha^2}{F^{2-2\beta}} + 
0.25 \cdot \rho \cdot \beta \cdot \nu \cdot \frac{\alpha}{F^{1-\beta}} + 
\frac{(2-3\rho^2)}{24} \cdot \nu^2\right) T \right]
$$

> NOT ATM condition

$$
\text{numer} =
\alpha \cdot \left[1 + \left(\frac{(1-\beta)^2}{24} \cdot \frac{\alpha^2}{(F K)^{1-\beta}} +
0.25 \cdot \rho \cdot \beta \cdot \nu \cdot \frac{\alpha}{(F K)^{(1-\beta)/2}} +
\frac{(2-3\rho^2)}{24} \cdot \nu^2\right) T \right] \cdot z
$$

where

$$
z = \frac{\nu}{\alpha} \cdot (F K)^{\frac{1 - \beta}{2}} \cdot \ln\left(\frac{F}{K}\right)
$$

and

$$
\chi = \ln\left(\frac{\sqrt{1 - 2\rho z + z^2} + z - \rho}{1 - \rho}\right)
$$

$$
\text{denom} = (F K)^{\frac{1-\beta}{2}} \cdot \left[1 + \frac{(1-\beta)^2}{24} \cdot (\ln(F/K))^2 + 
\frac{(1-\beta)^4}{1920} \cdot (\ln(F/K))^4\right] \cdot \chi
$$

$$
\sigma_{\text{imp}} = \frac{\text{numer}}{\text{denom}}
$$

In [231]:
# SABR implied volatility function
def sabr_volatility(F, K, T, alpha, beta, rho, nu):
    if np.isclose(F, K):  # Handle ATM case
        numer1 = ((1 - beta) ** 2 / 24) * (alpha ** 2) / (F ** (2 - 2 * beta))
        numer2 = 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))
        numer3 = ((2 - 3 * rho ** 2) / 24) * nu ** 2
        return alpha * (1 + (numer1 + numer2 + numer3) * T) / (F ** (1 - beta))
    else:
        z = (nu / alpha) * ((F * K) ** ((1 - beta) / 2)) * np.log(F / K)
        chi = np.log((np.sqrt(1 - 2 * rho * z + z ** 2) + z - rho) / (1 - rho))
        numer = alpha * (1 + (((1 - beta) ** 2 / 24) * (alpha ** 2) / ((F * K) ** (1 - beta)) +
                              0.25 * rho * beta * nu * alpha / ((F * K) ** ((1 - beta) / 2)) +
                              ((2 - 3 * rho ** 2) / 24) * nu ** 2) * T) * z
        denom = ((F * K) ** ((1 - beta) / 2)) * (1 + (1 - beta) ** 2 / 24 * (np.log(F / K)) ** 2 +
                                                 (1 - beta) ** 4 / 1920 * (np.log(F / K)) ** 4) * chi
        return numer / denom


# Black-Scholes call and put pricing
def black_scholes_call(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)


def black_scholes_put(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


# SABR-based call and put pricing
def sabr_call_price(S, K, r, T, alpha, beta, rho, nu):
    fwd = S * np.exp(r * T)
    sigma = sabr_volatility(fwd, K, T, alpha, beta, rho, nu)
    return black_scholes_call(S, K, r, sigma, T)


def sabr_put_price(S, K, r, T, alpha, beta, rho, nu):
    fwd = S * np.exp(r * T)
    sigma = sabr_volatility(fwd, K, T, alpha, beta, rho, nu)
    return black_scholes_put(S, K, r, sigma, T)


# Integration for static replication
def sabr_call_integrand(K, S, r, T, alpha, beta, rho, nu):
    weight = (-2 / 9) * K ** (-5 / 3) - 1.5 / K ** 2
    return sabr_call_price(S, K, r, T, alpha, beta, rho, nu) * weight


def sabr_put_integrand(K, S, r, T, alpha, beta, rho, nu):
    weight = (-2 / 9) * K ** (-5 / 3) - 1.5 / K ** 2
    return sabr_put_price(S, K, r, T, alpha, beta, rho, nu) * weight


# Static replication price calculation
def static_replication_price(S, r, T, alpha, beta, rho, nu, integration_limits):
    fwd_price = S * np.exp(r * T)
    h_F = (fwd_price ** (1 / 3)) + 1.5 * np.log(fwd_price) + 10

    put_integral = quad(lambda K: sabr_put_integrand(K, S, r, T, alpha, beta, rho, nu), integration_limits[0], fwd_price)[0]
    call_integral = quad(lambda K: sabr_call_integrand(K, S, r, T, alpha, beta, rho, nu), fwd_price, integration_limits[1])[0]

    return np.exp(-r * T) * h_F + put_integral + call_integral


# parameters
spx_params = {"alpha": 1.81600099, "beta": 0.7, "rho": -0.40141522, "nu": 2.79103083}
spy_params = {"alpha": 0.90783421, "beta": 0.7, "rho": -0.48729513, "nu": 2.72885668}
S1 = 3662.45
r = 0.01  # Example risk-free rate
T = 45/365  # Example time to maturity (in years)
integration_limits = (1e-6, 5000)
S2 =366.20
# Calculate static replication prices
spx_price = static_replication_price(S1, r, T, **spx_params, integration_limits=integration_limits)
spy_price = static_replication_price(S2, r, T, **spy_params, integration_limits=integration_limits)

print(f"SPX Static Replication Price: {spx_price:.9f}")
print(f"SPY Static Replication Price: {spy_price:.9f}")

SPX Static Replication Price: 37.669990999
SPY Static Replication Price: 25.972922929


## 2. “Model-free” integrated variance

$$
\sigma_{\text{MF}}^2 T = \mathbb{E} \left[ \int_0^T \sigma_t^2 \, dt \right]
$$

### Black-Scholes

$$
E\left[\int_0^T \sigma_t^2 dt\right] = 2e^{rT} \left( \int_0^F \frac{P(K)}{K^2} dK + \int_F^\infty \frac{C(K)}{K^2} dK \right)
$$

In [309]:
# spx

# Black-Scholes Put price function
def bs_put_price(K, S, r, sigma, T):
    from scipy.stats import norm

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

# Black-Scholes Call price function
def bs_call_price(K, S, r, sigma, T):
    from scipy.stats import norm

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# Function for the put integral
def put_integral(K, S, r, sigma, T):
    return bs_put_price(K, S, r, sigma, T) / (K**2)

# Function for the call integral
def call_integral(K, S, r, sigma, T):
    return bs_call_price(K, S, r, sigma, T) / (K**2)

# Parameters
S = 3662.45  # Spot price
r = r2  # Risk-free rate
sigma = sigma2x  # Volatility
T = 45/365  # Time to maturity
F = S * np.exp(r * T)  # Forward price

# Compute the put integral (from 0 to F)
I_put, _ = quad(put_integral, 0, F, args=(S, r, sigma, T))

# Compute the call integral (from F to infinity)
I_call, _ = quad(call_integral, F, np.inf, args=(S, r, sigma, T))

# Combine the results
integrated_variance = 2 * np.exp(r * T) * (I_put + I_call)
print(f"(X)Integrated Variance: {integrated_variance:.6f}")


(X)Integrated Variance: 0.004237


In [305]:
# spy

# Black-Scholes Put price function
def bs_put_price(K, S, r, sigma, T):
    from scipy.stats import norm

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

# Black-Scholes Call price function
def bs_call_price(K, S, r, sigma, T):
    from scipy.stats import norm

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# Function for the put integral
def put_integral(K, S, r, sigma, T):
    return bs_put_price(K, S, r, sigma, T) / (K**2)

# Function for the call integral
def call_integral(K, S, r, sigma, T):
    return bs_call_price(K, S, r, sigma, T) / (K**2)

# Parameters
S = 366.02  # Spot price
r = 0.01  # Risk-free rate
sigma = sigma3x  # Volatility
T = 45/365  # Time to maturity
F = S * np.exp(r * T)  # Forward price

# Compute the put integral (from 0 to F)
I_put, _ = quad(put_integral, 0, F, args=(S, r, sigma, T))

# Compute the call integral (from F to infinity)
I_call, _ = quad(call_integral, F, np.inf, args=(S, r, sigma, T))

# Combine the results
integrated_variance = 2 * np.exp(r * T) * (I_put + I_call)
print(f"(Y)Integrated Variance: {integrated_variance:.6f}")


(Y)Integrated Variance: 0.004496


### Bachelier

In [253]:
T = 45/365
spx_S = 3662.45
spy_S = 366.02
r = r2
spx_K = 3660
spy_K = 366

In [255]:
# Bachelier Model Integrated Variance Function
def Bach_callintegrand(K, S, r, T, sigma):
    price = BachelierCall(S, K, r, sigma, T) / K**2
    return price

def Bach_putintegrand(K, S, r, T, sigma):
    price = BachelierPut(S, K, r, sigma, T) / K**2
    return price


spx_F = spx_S * np.exp(r * T)
spx_I_put = quad(lambda x: Bach_putintegrand(x, spx_S, r, T, sigma2x), 0.0, spx_F)
spx_I_call = quad(lambda x: Bach_callintegrand(x, spx_S, r, T, sigma2x), spx_F, 5000)
spx_Bach_E_var = 2 * np.exp(r * T) * (spx_I_put[0] + spx_I_call[0])
print('The Bachelier expected integrated variance for SPX is: %.9f' % spx_Bach_E_var)


spy_F = spy_S * np.exp(r * T)
spy_I_put = quad(lambda x: Bach_putintegrand(x, spy_S, r, T, sigma2y), 0.0, spy_F)
spy_I_call = quad(lambda x: Bach_callintegrand(x, spy_S, r, T, sigma2y), spy_F, 5000)
spy_Bach_E_var = 2 * np.exp(r * T) * (spy_I_put[0] + spy_I_call[0])
print('The Bachelier expected integrated variance for SPY is: %.9f' % spy_Bach_E_var)

The Bachelier expected integrated variance for SPX is: 0.004263876
The Bachelier expected integrated variance for SPY is: 0.004237972


### SABR

In [281]:
def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) /(K ** 2)
    return price


def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / (K ** 2)
    return price
    
spx_alpha = 1.81600099
beta = 0.7
spx_rho = -0.40141522
spx_nu = 2.79103083

spy_alpha = 0.90783421
spy_rho = -0.48729513
spy_nu = 2.72885668

spx_F = spx_S * np.exp(r * T)

spx_I_put = quad(lambda x: sabrputintegrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), 1e-6, spx_F)
spx_I_call = quad(lambda x: sabrcallintegrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), spx_F, 5000)
spx_E_var = 2 * np.exp(r * T) * (spx_I_put[0] + spx_I_call[0])
print('The static replication expected integrated variance for SPX is: %.9f' % spx_E_var)

spy_F = spy_S * np.exp(r * T)
spy_I_put = quad(lambda x: sabrputintegrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), 1e-6, spy_F)
spy_I_call = quad(lambda x: sabrcallintegrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), spy_F, 5000)
spy_E_var = 2 * np.exp(r * T) * (spy_I_put[0] + spy_I_call[0])
print('The static replication expected integrated variance for SPY is: %.9f' % spy_E_var)

The static replication expected integrated variance for SPX is: 0.006333770
The static replication expected integrated variance for SPY is: 0.006013238
